<a href="https://colab.research.google.com/github/kyochanpy/forest_bears/blob/main/nb_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py
!pip install geopandas
!pip install jismesh
!pip install folium

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 15.4 MB 18.4 MB/s 
     |████████████████████████████████| 6.3 MB 46.6 MB/s 


In [ ]:
import re

import pandas as pd
import numpy as np
from tqdm import tqdm
import tabula
from tabula import read_pdf
import math
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime as dt

import geopandas as gpd
import jismesh.utils as ju
import folium
from shapely.geometry import Polygon
import branca.colormap as cm

In [ ]:
# エクセルを読み込んだ時に1行目がcolumnsになるのを修正する関数。
def set_columns(input_df):
    input_df_ = input_df.copy()

    output_df = input_df_.shift()                
    output_df.iloc[0] = output_df.columns.values
    output_df.columns = ['No.', '区', '発生日時', '出没場所', '状況']

    return output_df

In [ ]:
# 列がずれてNaNになるセルがあるのを修正する関数。
def fix_df(input_df):
    input_df_ = input_df.copy()
    
    idx_list = []
    no_list = input_df_["No."].tolist()
    for i in no_list:
        if np.isnan(i):
            idx_list.append(no_list.index(i))

    for i in idx_list:
        i_ = input_df_.iloc[i, 2]
        if type(i_) != float:
            input_df_.iloc[i, 2]  = i_.strftime('%H:%M')

    input_df_ = input_df_.fillna("")

    for i in idx_list:
        input_df_.iloc[i-1, 2:4] = input_df_.iloc[i-1, 2:4] + input_df_.iloc[i, 2:4]
        
    output_df = input_df_.drop(idx_list)

    return output_df

In [ ]:
# excelシートからdfを出力する関数。
def make_df_from_excel():
    output_df = pd.DataFrame()
    for i in range(5):
        excel_df = pd.read_excel("/content/drive/MyDrive/forest_bears/forest_bears.xlsx", sheet_name=i)
        tmp_df = set_columns(excel_df)
        df = fix_df(tmp_df)
        output_df = pd.concat([output_df, df])
    
    return output_df

In [ ]:
#ここでGoogle Map APIを使う。

In [ ]:
# Google Map APIで座標を追加する関数。
def get_latlng_features(input_df, key):
    input_df_ = input_df.copy()

    gmaps = googlemaps.Client(key=key)

    lat_list = []
    lng_list = []
    c = 0
    for i in input_df_["出没場所"].values:
        gmap_list = gmaps.geocode(i)
        try:
            latlng = gmap_list[0]["geometry"]["location"]
            lat = latlng["lat"]
            lng = latlng["lng"]
            lat_list.append(lat)
            lng_list.append(lng)
            c += 1
            if c%10 == 0:
                print(c)
        except IndexError:
            lat_list.append("不明")
            lng_list.append("不明")
            c += 1
            if c%10 == 0:
                print(c)

    output_df = input_df.copy()
    output_df["lat"] = lat_list
    output_df["lng"] = lng_list

    return output_df

In [ ]:
latlng_df = pd.read_csv('/content/drive/MyDrive/forest_bears/forest_bears_with_latlng.csv')

In [ ]:
latlng_df.head(200)

,No.,区,発生日時,出没場所,状況,lat,lng
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064
...,...,...,...,...,...,...,...
195,59.0,南区,6月28日(木),南区簾舞373番地付近,フンを確認,42.955114,141.265783
196,60.0,南区,6月28日(木),南区簾舞465番地付近,枝折り、足跡、爪跡、フン、被毛を確認,42.959331,141.253260
197,61.0,南区,6月29日(金),南区藤野472番地付近,フンを確認,42.937202,141.295218
198,62.0,南区,7月1日(日)14:00,南区芸術の森1丁目付近,ヒグマを目撃,42.945646,141.340777


In [ ]:
m = re.findall(r'\d+', "令和2年12月23日(水)18:30")

In [ ]:
s = "令和2年12月23日(水)18:30"

In [ ]:
p = r'\((.*)\)'  # アスタリスクに囲まれている任意の文字（アスタリスクを除く）
r = re.findall(p, s)
print(r)  # ['sample string']

['水']


In [ ]:
r[0]

'水'

In [ ]:
）"令和2年12月23日(水)18:30"[4:]

SyntaxError: ignored

In [ ]:
# 年度を追加する関数。
# 年度が含まれないデータがあるため、"No."列を利用する
def get_year_features(input_df):
    input_df_ = input_df.copy()

    no_values = input_df_["No."].values
    len_list_ = []
    for i in range(len(no_values)+1):
        if no_values[i+1] - no_values[i] != 1:
            len_list_.append(i)
            if len(len_list_) == 4:
                break
    len_list_.append(len(no_values))
    
    len_list = []
    for i in range(0,len(len_list_)):
        i_ = len_list_[i] - len_list_[i-1]
        len_list.append(i_)
    len_list[0] = len_list_[0]

    fiscal_year_list = []
    for i, j in zip(
        len_list,
        [2016, 2017, 2018, 2019, 2020]
    ):
        fiscal_year_list = fiscal_year_list + [j] * i
    
    output_df = input_df.copy()
    output_df["年度"] = fiscal_year_list

    return output_df

In [ ]:
# 年度以外を追加する関数。
def get_datetime_features(input_df):
    input_df_ = input_df.copy()

    datetime_list = []
    p = r'\((.*)\)'  # ()に囲まれている部分
    for d in input_df_["発生日時"]:
        if d[0] == "平" or d[0] == "令":
            d = d[3:]
        d_list = re.findall(r'\d+', d)
        w_day =  re.findall(p, d)[0]
        if len(d_list) == 2:
            datetime = [int(d_list[0]), int(d_list[1]), w_day, "不明"]
            datetime_list.append(datetime)
        else:
            datetime = [int(d_list[0]), int(d_list[1]), w_day, f"{d_list[2]}:{d_list[3]}"]
            datetime_list.append(datetime)
    datetime_df = pd.DataFrame(datetime_list)
    datetime_df.columns = ["月", "日", "曜日", "時間"]

    year_list = []
    for w_y, m in zip(
        input_df_["年度"].to_numpy(),
        datetime_df["月"].to_numpy()
    ):
        if 1 <= m <= 3:
            year_list.append(w_y+1)
        else:
            year_list.append(w_y)
    year_df = pd.DataFrame(year_list)
    year_df.columns = ["年"]

    output_df = pd.concat([input_df_, year_df, datetime_df], axis=1).drop("発生日時", axis=1)

    return output_df

In [ ]:
def get_features(input_df, funcs):
    output_df = input_df.copy()
    
    for func in funcs:
        output_df = func(output_df)

    return output_df

In [ ]:
get_features(latlng_df, [get_year_features, get_datetime_features])

,No.,区,出没場所,状況,lat,lng,年度,年,月,日,曜日,時間
0,1.0,南区,南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965,2016,2016,4,14,木,不明
1,2.0,南区,南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236,2016,2016,4,20,水,不明
2,3.0,南区,南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829,2016,2016,4,21,木,不明
3,4.0,南区,南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751,2016,2016,4,28,木,不明
4,5.0,南区,南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064,2016,2016,5,14,土,不明
...,...,...,...,...,...,...,...,...,...,...,...,...
558,91.0,手稲区,手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.077052,141.200096,2020,2020,11,12,木,不明
559,92.0,西区,西区小別沢20番地付近,足跡、被毛を確認,43.047359,141.277738,2020,2020,12,4,金,不明
560,93.0,南区,南区石山1045番地2付近,ヒグマを目撃,42.946191,141.328557,2020,2020,12,23,水,18:30
561,94.0,南区,南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.283236,2020,2021,3,25,木,不明


In [ ]:
def merge_weather_df(path_list):
    output_df = pd.DataFrame()

    for path in path_list:
        df = pd.read_csv(path, encoding="SHIFT-JIS")
        df_ = df.shift()
        df_.iloc[0] = df_.columns.values
        df_.columns = ["年", "月", "日", "平均気温", "日照時間", "日照ダミー", "合計全天日射量", "合計降雪量", "降雪ダミー", 
                       "平均風速", "最多風向", "平均蒸気圧", "平均現地気圧", "平均湿度", "平均雲量",
                       "天候（昼）", "天候（夜）", "最深積雪", "積雪ダミー", "合計降水量", "降水ダミー"]
        output_df = pd.concat([output_df, df_])
    
    return output_df


def fix_weather_data(input_df):
    output_df = input_df.copy()

    # concat時に崩れたダミー特徴量を修正
    for i in ["日照ダミー", "降雪ダミー", "積雪ダミー", "降水ダミー"]:
        output_df[i] = [i_[0] for i_ in output_df[i].values.astype(str)]

    # columnsからshiftさせるときに崩れた部分を修正
    for i in ["月", "日", "合計降雪量"]:
        list_ = []
        for i_ in output_df[i]:
            if type(i_) == str:
                list_.append(int(i_[0]))
            else:
                list_.append(i_)
        output_df[i] = list_

    # object, floatをintに変換
    for c in ["年", "月", "日"]:
        output_df[c] = output_df[c].astype(int)
    
    return output_df


def check_weather_str(input_str):
    if "時々" in input_str:
        tmp_str = input_str.split("時々")[0]
    else:
        tmp_str = input_str

    if "一時" in tmp_str:
        tmp_str = tmp_str.split("一時")[0]
    else:
        tmp_str = tmp_str
    
    if "、" in tmp_str:
        tmp_str = tmp_str.split("、")[0]
    else:
        tmp_str = tmp_str

    if "\u3000" in tmp_str:
        output_str = tmp_str.replace("\u3000", "")
    else:
        output_str = tmp_str
    
    return output_str


def simple_weather(input_list):
    output_list = []
    for i in input_list:
        if "晴" in i:
            i_ = "晴"
        elif "雨" in i:
            i_ = "雨"
        elif "曇" in i:
            i_ = "曇"
        else:
            i_ = "雪"
        output_list.append(i_)
    
    return output_list


def make_weather_features(input_df):
    input_df_ = input_df.copy()

    output_df = input_df_[["年", "月", "日", "平均気温"]]
    for c in ["天候（昼）", "天候（夜）"]:
        w_1 = []
        w_2 = []
        for i in input_df_[c]:
            if "後" in i:
                i_1 = i.split("後")[0]
                i_2 = i.split("後")[1]
                w_1.append(check_weather_str(i_1))
                w_2.append(check_weather_str(i_2))
            else:
                w_1.append(check_weather_str(i))
                w_2.append(check_weather_str(i))
        w_1_ = []
        w_2_ = []
        for i_1_, i_2_ in zip(
            w_1,
            w_2
        ):
            if i_2_ == "":
                i_2_ = i_1_
            else:
                i_2_ = i_2_
            w_1_.append(i_1_)
            w_2_.append(i_2_)  
        output_df[f"{c}_1"] = simple_weather(w_1_)
        output_df[f"{c}_2"] = simple_weather(w_2_)

    return output_df

In [ ]:
# 天候のdfを返す関数。
# 
def get_weather_df(input_df, path):
    input_df_ = input_df.copy()

    all_weather = merge_weather_df(path)
    fixed_df = fix_weather_data(all_weather)
    weather_df = make_weather_features(fixed_df)
    output_df = pd.merge(output_df, weather_df, on=["年", "月", "日"])

    return output_df

In [ ]:
output_df = pd.merge(output_df, weather_features, on=["年", "月", "日"])

In [ ]:
get_weather_df(latlng_df, ["/content/drive/MyDrive/forest_bears/weather_2016-2017.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2018-2019.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2020-2021.csv"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



UnboundLocalError: ignored

In [ ]:
def get_features(input_df, funcs):
    output_df = input_df.copy()
    
    for func in funcs:
        output_df = func(output_df)

    return output_df

In [ ]:
# KEY_CODEを含むデータにKEY_CODEのPolygonを追加する関数
def get_geometry_features(input_df, path):
    input_df_ = input_df.copy()
    geo_df = gpd.read_file(path)

    geo_df["KEY_CODE"] = geo_df["KEY_CODE"].astype(int)

    output_df = pd.merge(input_df_, geo_df.iloc[:, [0,-1]], on="KEY_CODE", how="left")

    return output_df

In [ ]:
# 経緯度からその地点のn次地域メッシュコードを追加する関数。
def get_mesh_features(input_df):
    input_df_ = input_df.copy()

    mesh_list = []
    for lat, lng in zip(
        input_df_["lat"].to_numpy(),
        input_df_["lng"].to_numpy()
    ):
        mesh_code = ju.to_meshcode(lat, lng, 4)
        mesh_list.append(mesh_code)

    output_df = input_df.copy()
    output_df["KEY_CODE"] = mesh_list

    return output_df

In [ ]:
# KEY_CODE(4次)を含むデータからそのKEY_CODEの人口の特徴量を生成する関数
# 別の分析でも使うため1995∼2015で使えるようにする。
def get_population_mesh_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    y = path[-13:-9]  # 各データでメッシュ内総人口の列名が異なる 
    if y == "1995":
        p = "T000752001"
    elif y == "2000":
        p = "T000386001"
    elif y == "2005":
        p = "T000387001"
    elif y == "2010":
        p = "T000609001"
    elif y == "2015":
        p = "T000847001"

    output_df = df.loc[:, ("KEY_CODE", p)].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={p:"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)

    return output_df

In [ ]:
get_population_mesh_features("/content/drive/MyDrive/forest_bears/2015_6441.txt")

,KEY_CODE,人口
1,644100052,10
2,644100061,152
3,644100063,66
4,644100064,69
5,644100083,4
...,...,...
6876,644177933,7
6877,644177934,3
6878,644177943,13
6879,644177944,3


In [ ]:
# DataFrameをGeoDataFrameに変換する関数
def GDF(input_df):
    input_df_ = input_df.copy()
    output_df = gpd.GeoDataFrame(input_df_)
    return output_df

In [ ]:
# メッシュの中心座標を求める関数
def min_max_coordinates(input_df):
    input_df_ = input_df.copy()

    lng_center_list = []
    lat_center_list = []
    for n in input_df_["geometry"]:
        coordinates_list = list(zip(*n.exterior.coords.xy))
        lng_list_ = []
        lat_list_ = []
        for m in coordinates_list:
            lng_list_.append(m[0])
            lat_list_.append(m[1])
        lng_mean = sum(lng_list_) / len(lng_list_)
        lng_center_list.append(lng_mean)
        lat_mean = sum(lat_list_) / len(lat_list_)
        lat_center_list.append(lat_mean)

    output_df = input_df.copy()
    output_df["lng_center"] = lng_center_list
    output_df["lat_center"] = lat_center_list
    
    return output_df

In [ ]:
def make_mesh4_df(input_df, population_path, geo_path, weather_path_list):
    input_df_ = input_df.copy()

    #Geocodingがうまくいかなかったと思われる座標（No.97）を手動で修正。
    input_df_["lat"] = input_df_["lat"].replace(42.6033477, 43.0033312)
    input_df_["lng"] = input_df_["lng"].replace(140.6160604, 141.3402989)

    #地域メッシュコードを追加。
    input_df_mesh = get_mesh4_features(input_df_)

    #日時の特徴量。
    date_time_features = get_date_time_features(input_df_)

    #天候の特徴量。
    weather_features_ = fix_weather_data(merge_weather_df(weather_path_list))
    weather_features = make_weather_features(weather_features_)

    #人口の特徴量（"KEY_CODE"有り）。
    population_features = get_population_mesh4_features(population_path)
    
    #特徴量をくっつける。
    output_df = pd.concat([input_df_mesh, date_time_features], axis=1)
    output_df = pd.merge(output_df, population_features, on="KEY_CODE", how="left")
    for c in ["年", "月", "日"]:
        output_df[c] = output_df[c].astype(int)
    output_df = pd.merge(output_df, weather_features, on=["年", "月", "日"])
    output_df["人口"] = output_df[["人口"]].fillna(0).astype(int)
    
    #geometry特徴量をmerge。
    output_df = get_geometry_features(output_df, geo_path)

    return output_df

In [ ]:
df = make_mesh4_df(latlng_df, 
                       "/content/drive/MyDrive/forest_bears/2015_6441.txt", 
                       "/content/drive/MyDrive/forest_bears/MESH06441_4",
                       ["/content/drive/MyDrive/forest_bears/weather_2016-2017.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2018-2019.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2020-2021.csv"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:109: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
ｄｆ

,No.,区,発生日時,出没場所,状況,lat,lng,KEY_CODE,年度,年,月,日,曜日,時間,人口,平均気温,天候（昼）_1,天候（昼）_2,天候（夜）_1,天候（夜）_2,geometry
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965,644132841,2016,2016,4,14,木,不明,67,11,曇,曇,雨,雨,"POLYGON ((141.30625 42.98333, 141.30000 42.983..."
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236,644132822,2016,2016,4,20,水,不明,0,9.7,晴,曇,曇,晴,"POLYGON ((141.28750 42.98333, 141.28125 42.983..."
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829,644130733,2016,2016,4,21,木,不明,0,13.8,晴,曇,曇,曇,"POLYGON ((141.04375 42.97917, 141.03750 42.979..."
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751,644132872,2016,2016,4,28,木,不明,1417,7.6,曇,曇,雨,雨,"POLYGON ((141.35000 42.98333, 141.34375 42.983..."
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064,644131123,2016,2016,5,14,土,不明,0,14.5,晴,晴,曇,曇,"POLYGON ((141.15625 42.92917, 141.15000 42.929..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,91.0,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.077052,141.200096,644141961,2020,2020,11,12,木,不明,0,4.3,晴,晴,晴,曇,"POLYGON ((141.20625 43.07500, 141.20000 43.075..."
559,92.0,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認,43.047359,141.277738,644142523,2020,2020,12,4,金,不明,6,0.8,晴,曇,曇,曇,"POLYGON ((141.28125 43.04583, 141.27500 43.045..."
560,93.0,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃,42.946191,141.328557,644132363,2020,2020,12,23,水,18:30,41,1.3,晴,晴,曇,曇,"POLYGON ((141.33125 42.94583, 141.32500 42.945..."
561,94.0,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.283236,644132822,2020,2021,3,25,木,不明,0,9.5,曇,曇,雨,雨,"POLYGON ((141.28750 42.98333, 141.28125 42.983..."


In [ ]:
ｄｆdf.dtypes

No.          float64
区             object
発生日時          object
出没場所          object
状況            object
lat          float64
lng          float64
KEY_CODE       int64
年度             int64
年              int64
月              int64
日              int64
曜日            object
時間            object
人口             int64
平均気温          object
天候（昼）_1       object
天候（昼）_2       object
天候（夜）_1       object
天候（夜）_2       object
geometry    geometry
dtype: object

In [ ]:
all_df = pd.read_csv('/content/drive/MyDrive/forest_bears/all_df.csv')

In [ ]:
sapporo = pd.read_csv('/content/drive/MyDrive/forest_bears/only_sapporo_df.csv')

In [ ]:
all_df["geometry"] = all_df["geometry"].astype(geometry)

NameError: ignored

In [ ]:
population_df = GDF(get_population_mesh4_features("/content/drive/MyDrive/forest_bears/2015_6441.txt"))

In [ ]:
geo_df = GDF(get_geometry_features(population_df, "/content/drive/MyDrive/forest_bears/MESH06441_4"))

In [ ]:
lat, lng = (43.0,141.279425)
m = folium.Map((lat, lng), zoom_start=10.5)
folium.Choropleth(geo_data=geo_df, 
                    name='choropleth',
                    data=geo_df,
                    columns=['KEY_CODE', '人口'],
                    key_on='feature.properties.KEY_CODE',
                    fill_color='YlGn',
                    fill_opacity=0.7,
                    line_opacity=0.2,
                    legend_name='population').add_to(m)

    

In [ ]:
m

In [ ]:
def plot_circle_with_choropleth(input_df, geo_df, color):
    lat, lng = (43.0,141.279425)
    m = folium.Map((lat, lng), zoom_start=10.5)
    folium.Choropleth(geo_data=geo_df, 
                        name='choropleth',
                        data=geo_df,
                        columns=['KEY_CODE', '人口'],
                        key_on='feature.properties.KEY_CODE',
                        fill_color='YlGn',
                        fill_opacity=0.7,
                        line_opacity=0.2,
                        legend_name='population').add_to(m)

    for i in range(len(input_df)):
        folium.Circle(
            location=[input_df.iloc[i]['lat'], input_df.iloc[i]['lng']],
            radius=200,
            fill=True,
            color=color,
            fill_opacity=1.0
        ).add_to(m)

    return m

In [ ]:
plot_circle_with_choropleth(all_df[all_df["年度"] == 2016], geo_df, "blue")